# Solar Energy Consumption and Production Analysis
This notebook analyzes solar energy consumption and production. It calculates various metrics, including PV production, unmet demand, and performance indicators, under different conditions (e.g., number of panels and batteries).



### Solar Energy Consumption and Production Analysis for Ense3 building (For one Week)

In [ ]:
# --------------------------------
# Required Library Installation
# --------------------------------

import sys
import subprocess

# List of required Python libraries for the project
required_libraries = [
    'pandas',            # Data processing and manipulation
    'numpy',             # Numerical calculations
    'plotly',            # Interactive visualizations
    'ipywidgets',        # Interactive widgets for user input
    'matplotlib',        # Plotting capabilities 
]

# Function to check and install missing libraries
def install_missing_libraries(libraries):
    for lib in libraries:
        try:
            __import__(lib)
        except ImportError:
            print(f"Installing missing library: {lib}")
            subprocess.check_call([sys.executable, "-m", "pip", "install", lib])
        else:
            print(f"✔ {lib} is already installed.")

# Run the function to install missing libraries
install_missing_libraries(required_libraries)


In [2]:
# Cell 2: Importing Modules and Reloading solar_analysis_script
import os
import sys
import plotly.graph_objects as go
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output
import solar_analysis_script as sas
import datetime

# Reload the updated solar_analysis_script module in case of changes
import importlib
importlib.reload(sas)

# Cell 3: Define File Paths and Parameters

# Define file paths
consumption_file = 'Ense3buildingconsumption.csv'
pv_file = 'Ense3buildingPV.csv'

# Define separate date ranges
consumption_start_date = '2023-09-25 00:00:00'
consumption_end_date = '2024-09-25 00:00:00'

# Specify the actual column names in your consumption CSV
consumption_time_col = 'Time'
consumption_value_col = 'Consumption (kWh)'

# Number of solar panels (initial default value; can be adjusted via widget)
num_panels = 100  # Example value; adjust as needed

# Cell 4: Load and Merge Data

# Load and Merge Data

try:
    merged_data = sas.load_data(
        consumption_file, 
        pv_file, 
        consumption_start_date, 
        consumption_end_date,
        consumption_sep=';', 
        consumption_time_col=consumption_time_col, 
        consumption_value_col=consumption_value_col,
        pv_sep=','
    )
    print("Data successfully loaded and merged after cleaning.")
except Exception as e:
    print(f"An error occurred while loading data: {e}")

# Ensure index is a DatetimeIndex
merged_data.index = pd.to_datetime(merged_data.index)

# Cell 5: Define System Parameters

# Define System Parameters
battery_capacity_per_unit = 57  # Example: 57 kWh per battery
converter_efficiency = 0.9    # 90% converter efficiency
initial_soc = 50              # 50% initial state of charge on the first day
battery_soc_tracking = {}     # Initialize empty dict to track SOC for each day

# Cell 6: Create Interactive Widgets

# Create Interactive Widgets

# Widgets
num_panels_slider = widgets.IntSlider(
    value=0, min=0, max=1500, step=10, 
    description='Num Panels:', continuous_update=False
)
num_batteries_slider = widgets.IntSlider(
    value=0, min=0, max=100, step=1, 
    description='Num Batteries:', continuous_update=False
)
month_dropdown = widgets.Dropdown(
    options=[(pd.to_datetime(str(month), format='%m').strftime('%B'), month) for month in range(1, 13)], 
    value=1, description='Month:'
)
week_dropdown = widgets.Dropdown(
    options=[], value=None, description='Week Start Date:'
)
message_output = widgets.Output()
plot_output = widgets.Output()

# Arrange widgets in a layout
widgets_box = widgets.VBox([
    widgets.HBox([num_panels_slider, num_batteries_slider]), 
    widgets.HBox([month_dropdown, week_dropdown]), 
    message_output,
])
display(widgets_box, plot_output)

# Cell 7: Populate Week Dropdown Based on Selected Month

# Populate Week Dropdown Based on Selected Month

def populate_week_dropdown(change):
    selected_month = change['new']
    # Get all unique dates in the selected month
    dates_in_month = merged_data[merged_data.index.month == selected_month].index.normalize().unique()
    # Sort the dates
    dates_in_month = np.sort(dates_in_month)
    
    # Generate list of week start dates
    week_start_dates = []
    seen_dates = set()
    for date in dates_in_month:
        # Convert numpy.datetime64 to pandas.Timestamp
        date = pd.Timestamp(date)
        if date in seen_dates:
            continue
        # Define a week as 7 consecutive days starting from 'date'
        week = [date + pd.Timedelta(days=i) for i in range(7)]
        # Check if all days in the week are present in the data
        if all(d in dates_in_month for d in week):
            week_start_dates.append(date.strftime('%Y-%m-%d'))
            # Mark these dates as seen
            seen_dates.update(week)
    
    with message_output:
        message_output.clear_output()
        if week_start_dates:
            week_dropdown.options = week_start_dates
            week_dropdown.value = week_start_dates[0]
            print(f"Available weeks in {pd.to_datetime(selected_month, format='%m').strftime('%B')}:")
            for ws in week_start_dates:
                print(f"Week starting on {ws}")
        else:
            week_dropdown.options = []
            week_dropdown.value = None
            print(f"No complete weeks available for {pd.to_datetime(selected_month, format='%m').strftime('%B')}.")
    
month_dropdown.observe(populate_week_dropdown, names='value')
populate_week_dropdown({'new': month_dropdown.value})

# Cell 8: Define the Interactive Graph Update Function

# Define the Interactive Graph Update Function

def update_graph(num_panels, num_batteries, selected_week_start):
    with plot_output:
        plot_output.clear_output()  # Clear previous plot
        
        if not selected_week_start:
            with message_output:
                message_output.clear_output()
                print("[Error] No week selected. Please select a valid week.")
            return
        
        # Convert selected_week_start to Timestamp
        selected_week_start = pd.Timestamp(selected_week_start)
        selected_week_end = selected_week_start + pd.Timedelta(days=7)
        
        # Filter data for the selected week
        try:
            weekly_data = merged_data.loc[selected_week_start:selected_week_end - pd.Timedelta(hours=1)]
        except KeyError:
            with message_output:
                message_output.clear_output()
                print(f"[Error] No data available for the selected week starting on: {selected_week_start}")
            return

        if weekly_data.empty:
            with message_output:
                message_output.clear_output()
                print(f"[Warning] No data available for the selected week starting on: {selected_week_start}")
            return

        # Ensure weekly_data is a DataFrame
        if isinstance(weekly_data, pd.Series):
            weekly_data = weekly_data.to_frame().T

        # Calculate PV production for the selected week and number of panels
        weekly_pv_production = weekly_data['PV_Production_kWh'].values * num_panels 

        # Initialize battery parameters
        battery_capacity = battery_capacity_per_unit * num_batteries
        soc = initial_soc  # Starting SOC

        # Initialize lists to store metrics
        energy_charged = []
        energy_discharged = []
        unmet_demand = []
        excess_energy = []
        daily_consumption = []
        daily_pv = []
        energy_supplied_pv_to_demand = []  # New list for SSR

        # Iterate over each day in the week
        for day in range(7):
            current_day = selected_week_start + pd.Timedelta(days=day)
            try:
                daily_data = merged_data.loc[current_day.strftime('%Y-%m-%d')]
            except KeyError:
                # If no data for the day, append zeros
                energy_charged.extend([0]*24)
                energy_discharged.extend([0]*24)
                unmet_demand.extend([0]*24)
                excess_energy.extend([0]*24)
                daily_consumption.extend([0]*24)
                daily_pv.extend([0]*24)
                energy_supplied_pv_to_demand.extend([0]*24)
                continue

            if daily_data.empty:
                # If no data for the day, append zeros
                energy_charged.extend([0]*24)
                energy_discharged.extend([0]*24)
                unmet_demand.extend([0]*24)
                excess_energy.extend([0]*24)
                daily_consumption.extend([0]*24)
                daily_pv.extend([0]*24)
                energy_supplied_pv_to_demand.extend([0]*24)
                continue

            for hour in range(24):
                timestamp = current_day + pd.Timedelta(hours=hour)
                if timestamp not in daily_data.index:
                    consumption = 0
                    pv_production = 0
                else:
                    consumption = daily_data.loc[timestamp]['Consumption_kWh']
                    # To correctly index weekly_pv_production, calculate the relative hour
                    relative_hour = (timestamp - selected_week_start).total_seconds() // 3600
                    relative_hour = int(relative_hour)
                    if 0 <= relative_hour < len(weekly_pv_production):
                        pv_production = weekly_pv_production[relative_hour]
                    else:
                        pv_production = 0

                daily_consumption.append(consumption)
                daily_pv.append(pv_production)
                
                # Calculate energy supplied by PV to demand
                energy_supplied = min(pv_production, consumption)
                energy_supplied_pv_to_demand.append(energy_supplied)

                if pv_production >= consumption:
                    excess = pv_production - consumption
                    if num_batteries > 0 and soc < 100:
                        charge = min(excess * converter_efficiency, (100 - soc) / 100 * battery_capacity)
                        energy_charged.append(charge)
                        soc += (charge / battery_capacity) * 100
                        # Store excess energy as negative
                        excess_energy.append(-(excess - charge / converter_efficiency))
                    else:
                        energy_charged.append(0)
                        # Store excess energy as negative
                        excess_energy.append(-excess)
                    energy_discharged.append(0)
                    unmet_demand.append(0)
                else:
                    deficit = consumption - pv_production
                    if num_batteries > 0 and soc > 20:
                        discharge = min(deficit, (soc - 20) / 100 * battery_capacity * converter_efficiency)
                        energy_discharged.append(discharge)
                        soc -= (discharge / battery_capacity) * 100
                        unmet_demand.append(deficit - discharge)
                    else:
                        energy_discharged.append(0)
                        unmet_demand.append(deficit)
                    energy_charged.append(0)
                    excess_energy.append(0)

        # Create a multi-day plot
        fig = go.Figure()

        # Define colors for clarity
        colors = {
            'Solar PV Production (kWh)': 'orange',
            'Unmet Demand (kWh)': 'red',
            'Energy Charged to Battery (kWh)': 'green',
            'Energy Discharged from Battery (kWh)': 'purple',
            'Excess Energy (kWh)': 'cyan',
            'Electricity Demand (kWh)': 'blue'
        }

        # Create x-axis labels for the week (Day Hour)
        x_labels = []
        for day in range(7):
            for hour in range(24):
                date = selected_week_start + pd.Timedelta(days=day, hours=hour)
                label = date.strftime('%a %d-%b %H:%M')
                x_labels.append(label)

        # Add traces
        fig.add_trace(go.Bar(
            x=x_labels, y=weekly_pv_production, name='Solar PV Production (kWh)', marker_color=colors['Solar PV Production (kWh)'],
            hovertemplate='Solar PV Production: %{y} kWh<extra></extra>'
        ))
        fig.add_trace(go.Bar(
            x=x_labels, y=unmet_demand, name='Unmet Demand (kWh)', marker_color=colors['Unmet Demand (kWh)'],
            hovertemplate='Unmet Demand: %{y} kWh<extra></extra>'
        ))
        if num_batteries > 0:
            fig.add_trace(go.Bar(
                x=x_labels, y=[-val for val in energy_charged], name='Energy Charged to Battery (kWh)', marker_color=colors['Energy Charged to Battery (kWh)'],
                hovertemplate='Energy Charged to Battery: %{y} kWh<extra></extra>'
            ))
            fig.add_trace(go.Bar(
                x=x_labels, y=energy_discharged, name='Energy Discharged from Battery (kWh)', marker_color=colors['Energy Discharged from Battery (kWh)'],
                hovertemplate='Energy Discharged from Battery: %{y} kWh<extra></extra>'
            ))
        fig.add_trace(go.Bar(
            x=x_labels, y=excess_energy, name='Excess Energy (kWh)', marker_color=colors['Excess Energy (kWh)'],
            hovertemplate='Excess Energy: %{y} kWh<extra></extra>'
        ))
        fig.add_trace(go.Scatter(
            x=x_labels, y=daily_consumption, mode='lines', name='Electricity Demand (kWh)', line=dict(color=colors['Electricity Demand (kWh)'], dash='dash'),
            hovertemplate='Electricity Demand: %{y} kWh<extra></extra>'
        ))

        # Add date annotations below the x-axis using 'paper' coordinates
        annotations = []
        annotation_y = -0.15  # Fixed y position relative to the paper; adjust as needed

        for day in range(7):
            # Calculate the position (index) where each day starts
            start_idx = day * 24
            # Calculate the center position for the annotation
            center_idx = start_idx + 12  # Middle of the day
            date_label = (selected_week_start + pd.Timedelta(days=day)).strftime('%Y-%m-%d')
            annotations.append(dict(
                x=x_labels[center_idx],
                y=annotation_y,  # Fixed y position relative to the paper
                xref='x',
                yref='paper',
                text=date_label,
                showarrow=False,
                font=dict(
                    size=12,
                    color='black'
                ),
                xanchor='center',
                yanchor='top'
            ))

        # Update layout for better readability
        fig.update_layout(
            title=f'Solar PV Production, Battery Management, and Electricity Demand from {selected_week_start.strftime("%Y-%m-%d")} to {(selected_week_start + pd.Timedelta(days=6)).strftime("%Y-%m-%d")}',
            xaxis_title='Date and Hour',
            yaxis_title='Energy (kWh)',
            template='plotly_white',
            barmode='relative',
            height=700,  # Increased height to accommodate annotations
            xaxis=dict(
                tickangle=-45,
                tickfont=dict(size=10),
                tickvals=[24*day for day in range(7)],  # Set tick positions at the start of each day
                ticktext=['']*7  # Hide existing tick labels
            ),
            annotations=annotations,
            margin=dict(b=150)  # Increased bottom margin to prevent clipping of annotations
        )

        fig.show()

        # Calculate total energy supplied by PV to demand
        total_energy_supplied_pv = sum(energy_supplied_pv_to_demand)

        # Calculate total energy demand
        total_energy_demand = sum(daily_consumption)

        # Calculate Self-Sufficiency Rate (SSR)
        ssr = ((total_energy_supplied_pv + sum(energy_discharged))/ total_energy_demand) * 100 if total_energy_demand > 0 else 0

        # Optionally, display summary metrics
        with message_output:
            message_output.clear_output()
            total_pv = sum(weekly_pv_production)
            total_unmet = sum(unmet_demand)
            total_charged = sum(energy_charged)
            total_discharged = sum(energy_discharged)
            total_excess = sum(excess_energy)
            total_consumption = sum(daily_consumption)
            print("Weekly Summary Metrics:")
            print(f"Total PV Produced: {total_pv:.1f} kWh")
            print(f"Total Energy Charged to Battery: {total_charged:.1f} kWh")
            print(f"Total Energy Discharged from Battery: {total_discharged:.1f} kWh")
            print(f"Total Unmet Demand: {total_unmet:.1f} kWh")
            print(f"Total Excess Energy: {total_excess:.1f} kWh")
            print(f"Total Energy Supplied by PV to Demand: {total_energy_supplied_pv:.1f} kWh")
            print(f"Total Energy Demand: {total_energy_demand:.1f} kWh")
            print(f"Self-Sufficiency Rate (SSR): {ssr:.1f}%")

# Cell 9: Create Interactive Output for Weekly Plot

# Create the interactive output for weekly plot
out = widgets.interactive_output(
    update_graph,
    {
        'num_panels': num_panels_slider, 
        'num_batteries': num_batteries_slider, 
        'selected_week_start': week_dropdown
    }
)

# Display the interactive output
display(out)

INFO:solar_analysis_script:Loading energy consumption data from file: Ense3buildingconsumption.csv


INFO:solar_analysis_script:Energy consumption data successfully loaded and filtered.
INFO:solar_analysis_script:Loading PV production data from file: Ense3buildingPV.csv
INFO:solar_analysis_script:'Time_End' column dropped from PV production data.
INFO:solar_analysis_script:PV production data successfully loaded, converted to kWh, and processed.
INFO:solar_analysis_script:Extending PV production data to cover 2023-09-25 00:00:00 to 2024-09-25 00:00:00.
INFO:solar_analysis_script:Original PV data duration: 364 days 23:00:00 (8760 hours)
INFO:solar_analysis_script:Total duration to cover: 366 days 00:00:00 (8784 hours)
INFO:solar_analysis_script:Repeating PV data 2 times to cover the desired period.
INFO:solar_analysis_script:PV production data extended from 2023-09-25 00:00:00 to 2024-09-24 23:00:00.
INFO:solar_analysis_script:Total PV production records after extension: 8784
INFO:solar_analysis_script:'Consumption (kWh)' column dropped from merged data.
INFO:solar_analysis_script:'Elec

Data successfully loaded and merged after cleaning.


Output()

Output()